# Pytorch練習
- 基礎

In [18]:
import torch
print(tr.__version__)

1.0.1


In [20]:
list = [[1,2,3],[4,5,6]]
x = torch.Tensor(list)
print(x.size())

torch.Size([2, 3])


In [21]:
y = torch.randn(2,2)
print(y)
z = torch.eye(3,3)
print(z)
n = torch.linspace(10,100,10)
print(n)
print(torch.mm(x,z))

tensor([[0.5217, 0.5939],
        [0.0648, 0.2491]])
tensor([[1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.]])
tensor([ 10.,  20.,  30.,  40.,  50.,  60.,  70.,  80.,  90., 100.])
tensor([[1., 2., 3.],
        [4., 5., 6.]])


- ディープラーニング実装

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0, ), (1, ))])#mnistデータの正規化
trainset = torchvision.datasets.MNIST(root='./data', 
                                        train=True,
                                        download=True,
                                        transform=transform)
trainloader = torch.utils.data.DataLoader(trainset,
                                            batch_size=100,
                                            shuffle=True,#各epochでシャッフル
                                            num_workers=2)#並列処理の数
testset = torchvision.datasets.MNIST(root='./data', 
                                        train=False, 
                                        download=True, 
                                        transform=transform)
testloader = torch.utils.data.DataLoader(testset, 
                                            batch_size=100,
                                            shuffle=False, 
                                            num_workers=2)
classes = tuple(np.linspace(0, 9, 10, dtype=np.uint8))#処理が早いが変更できない

In [2]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3) # 28x28x1 -> 26x26x32
        self.conv2 = nn.Conv2d(32, 64, 3) # 26x26x32 -> 24x24x64 
        self.pool = nn.MaxPool2d(2, 2) # 24x24x64 -> 12x12x64
        self.dropout1 = nn.Dropout2d()
        self.fc1 = nn.Linear(12 * 12 * 64, 128)
        self.dropout2 = nn.Dropout2d()
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.dropout1(x)
        x = x.view(-1, 12 * 12 * 64)
        x = F.relu(self.fc1(x))
        x = self.dropout2(x)
        x = self.fc2(x)
        return x

In [5]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [4]:
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dropout1): Dropout2d(p=0.5)
  (fc1): Linear(in_features=9216, out_features=128, bias=True)
  (dropout2): Dropout2d(p=0.5)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)


In [9]:
epochs = 20
for epoch in range(epochs):
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(trainloader, 0):
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 100 == 99:
            print('[{:d}, {:5d}] loss: {:.3f}'
                    .format(epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0

print('Finished Training')

[1,   100] loss: 0.508
[1,   200] loss: 0.498
[1,   300] loss: 0.441
[1,   400] loss: 0.416
[1,   500] loss: 0.416
[1,   600] loss: 0.394
[2,   100] loss: 0.378
[2,   200] loss: 0.364
[2,   300] loss: 0.349
[2,   400] loss: 0.324
[2,   500] loss: 0.321
[2,   600] loss: 0.315
[3,   100] loss: 0.318
[3,   200] loss: 0.306
[3,   300] loss: 0.270
[3,   400] loss: 0.308
[3,   500] loss: 0.276
[3,   600] loss: 0.283
[4,   100] loss: 0.281
[4,   200] loss: 0.259
[4,   300] loss: 0.268


KeyboardInterrupt: 

In [10]:
correct = 0
total = 0

with torch.no_grad():
    for (images, labels) in testloader:
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print('Accuracy: {:.2f} %%'.format(100 * float(correct/total)))

Accuracy: 92.58 %%
